In [5]:
from pymongo.mongo_client import MongoClient
import json
from tqdm import tqdm

In [6]:
def get_mongo_client(mongo_uri):
    client = MongoClient(mongo_uri)
    return client

In [3]:
mongo_client = get_mongo_client("mongodb://localhost:27018/?directConnection=true")
db = mongo_client.get_database("triplets_db_hotpot_gpt4_1_mini")

In [4]:
mongo_client.list_database_names()

['admin',
 'config',
 'demo',
 'local',
 'non_onto_triplets_db_gpt4_1_mini',
 'non_onto_triplets_db_gpt4_o_mini',
 'onto_triplets_db_gpt4o-mini',
 'triplets_db',
 'triplets_db_gpt4_1_mini',
 'triplets_db_hotpot_gpt4_1_mini',
 'wikidata_ontology',
 'wikidata_ontology_test']

In [7]:
db = mongo_client.get_database("triplets_db_gpt4_1_mini")
db.list_collection_names()

['filtered_triplets',
 'ontology_filtered_triplets',
 'initial_triplets',
 'triplets',
 'entity_aliases']

In [8]:
db.get_collection("triplets").find_one({})

{'_id': ObjectId('68cc7139d481a3884ea4a540'),
 'sample_id': '2hop__121145_561444',
 'subject': 'The Real L Word',
 'object': 'reality television series',
 'subject_type': 'television series',
 'object_type': 'genre',
 'relation': 'instance of',
 'completion_token_num': 20,
 'prompt_token_num': 497,
 'qualifiers': [],
 'source_text_id': 0}

In [9]:
# To list search indexes in a MongoDB collection using PyMongo:
list(db.get_collection("triplets").find({}, {'_id': 0, 'sample_id': 1, 'relation': 1, 'subject': 1, 'object': 1, 'subject_type': 1, 'object_type': 1, 'qualifiers': 1}))

[{'sample_id': '2hop__121145_561444',
  'subject': 'The Real L Word',
  'object': 'reality television series',
  'subject_type': 'television series',
  'object_type': 'genre',
  'relation': 'instance of',
  'qualifiers': []},
 {'sample_id': '2hop__121145_561444',
  'subject': 'The Real L Word',
  'object': 'United States',
  'subject_type': 'television series',
  'object_type': 'country',
  'relation': 'country of origin',
  'qualifiers': []},
 {'sample_id': '2hop__121145_561444',
  'subject': 'The Real L Word',
  'object': 'Showtime',
  'subject_type': 'television series',
  'object_type': 'television network',
  'relation': 'broadcast by',
  'qualifiers': []},
 {'sample_id': '2hop__121145_561444',
  'subject': 'The Real L Word',
  'object': 'June 20, 2010',
  'subject_type': 'television series',
  'object_type': 'calendar date',
  'relation': 'date of first performance',
  'qualifiers': []},
 {'sample_id': '2hop__121145_561444',
  'subject': 'The Real L Word',
  'object': 'Ilene Chai

In [17]:
result = db.get_collection("filtered_triplets").aggregate([
    {
        "$group": {
            "_id": {
                "$concat": [
                    "$sample_id",
                    "`_",
                    { "$toString": "$source_text_id" }
                ]
            },
            "count": {"$sum": 1}
        }
    }
])

filtered_triplets = {}
for doc in result:
    filtered_triplets[doc["_id"]] = doc["count"]

result = db.get_collection("ontology_filtered_triplets").aggregate([
    {
        "$group": {
            "_id": "$sample_id",
            "count": {"$sum": 1}
        }
    }
])

ontology_filtered_triplets = {}
for doc in result:
    ontology_filtered_triplets[doc["_id"]] = doc["count"]

ontology_filtered_triplets

{'4hop3__220945_88460_30152_20999': 5,
 '2hop__458672_20057': 3,
 '3hop1__178435_547811_41132': 8,
 '3hop1__823336_228453_86925': 6,
 '2hop__283_574195': 4,
 '3hop1__79039_131926_87157': 6,
 '3hop1__273571_834494_34099': 5,
 '2hop__276637_126089': 6,
 '2hop__16844_42173': 10,
 '3hop1__226638_538202_84283': 2,
 '2hop__418941_429437': 4,
 '2hop__35173_17335': 5,
 '2hop__567471_123283': 5,
 '3hop1__680374_503371_21711': 9,
 '4hop1__726391_153080_33952_33939': 21,
 '2hop__580458_139390': 1,
 '3hop1__773338_42197_18397': 7,
 '3hop2__87184_38738_76291': 11,
 '3hop1__379315_160713_77246': 7,
 '4hop1__131632_32392_823060_610794': 12,
 '2hop__511296_2684': 9,
 '3hop1__857_846_7846': 10,
 '2hop__403060_92763': 11,
 '3hop1__579562_629431_124169': 9,
 '2hop__3739_13529': 2,
 '2hop__96062_159673': 20,
 '2hop__91021_19033': 6,
 '3hop1__241085_42197_18397': 7,
 '2hop__512773_346751': 5,
 '3hop1__79462_91850_685675': 3,
 '3hop1__602329_792411_51423': 3,
 '2hop__145282_21711': 2,
 '3hop2__304722_591230

In [18]:
result = db.get_collection("initial_triplets").aggregate([
    # {
    #     "$group": {
    #         "_id": {
    #             "$concat": [
    #                 "$sample_id",
    #                 "_",
    #                 { "$toString": "$source_text_id" }
    #             ]
    #         },
    #         "count": {"$sum": 1}
    #     }
    # }
        {
        "$group": {
            "_id": "$sample_id",
            "count": {"$sum": 1}
        }
    }
])

initial_triplets = {}
for doc in result:
    initial_triplets[doc["_id"]] = doc["count"]

initial_triplets

{'4hop3__220945_88460_30152_20999': 225,
 '2hop__458672_20057': 168,
 '3hop1__178435_547811_41132': 184,
 '3hop1__823336_228453_86925': 182,
 '2hop__283_574195': 159,
 '3hop1__79039_131926_87157': 196,
 '3hop1__273571_834494_34099': 186,
 '2hop__276637_126089': 182,
 '2hop__16844_42173': 211,
 '3hop1__226638_538202_84283': 193,
 '2hop__418941_429437': 116,
 '2hop__35173_17335': 216,
 '2hop__567471_123283': 177,
 '3hop1__680374_503371_21711': 171,
 '4hop1__726391_153080_33952_33939': 212,
 '2hop__580458_139390': 225,
 '3hop1__773338_42197_18397': 220,
 '3hop2__87184_38738_76291': 189,
 '3hop1__379315_160713_77246': 228,
 '4hop1__131632_32392_823060_610794': 205,
 '2hop__511296_2684': 160,
 '3hop1__857_846_7846': 169,
 '2hop__403060_92763': 254,
 '3hop1__579562_629431_124169': 184,
 '2hop__3739_13529': 147,
 '2hop__96062_159673': 254,
 '2hop__91021_19033': 163,
 '3hop1__241085_42197_18397': 223,
 '2hop__512773_346751': 255,
 '3hop1__79462_91850_685675': 208,
 '2hop__123148_5385': 154,
 '

In [12]:
result = db.get_collection("triplets").aggregate([
    # {
    #     "$group": {
    #         "_id": {
    #             "$concat": [
    #                 "$sample_id",
    #                 "_",
    #                 { "$toString": "$source_text_id" }
    #             ]
    #         },
    #         "count": {"$sum": 1}
    #     }
    # }
        {
        "$group": {
            "_id": "$sample_id",
            "count": {"$sum": 1}
        }
    }
])

triplets = {}
for doc in result:
    triplets[doc["_id"]] = doc["count"]

triplets

{'4hop3__220945_88460_30152_20999': 214,
 '2hop__458672_20057': 154,
 '3hop1__178435_547811_41132': 172,
 '3hop1__823336_228453_86925': 173,
 '2hop__283_574195': 148,
 '3hop1__79039_131926_87157': 181,
 '3hop1__273571_834494_34099': 181,
 '2hop__276637_126089': 172,
 '2hop__16844_42173': 199,
 '3hop1__226638_538202_84283': 183,
 '2hop__418941_429437': 112,
 '2hop__35173_17335': 197,
 '2hop__567471_123283': 164,
 '3hop1__680374_503371_21711': 160,
 '4hop1__726391_153080_33952_33939': 180,
 '2hop__580458_139390': 220,
 '3hop1__773338_42197_18397': 206,
 '3hop2__87184_38738_76291': 171,
 '3hop1__379315_160713_77246': 214,
 '4hop1__131632_32392_823060_610794': 188,
 '2hop__511296_2684': 149,
 '3hop1__857_846_7846': 156,
 '2hop__403060_92763': 238,
 '3hop1__579562_629431_124169': 171,
 '2hop__3739_13529': 142,
 '2hop__96062_159673': 232,
 '2hop__91021_19033': 154,
 '3hop1__241085_42197_18397': 212,
 '2hop__512773_346751': 245,
 '3hop1__79462_91850_685675': 201,
 '2hop__123148_5385': 147,
 '

In [13]:
len(triplets)

200

In [15]:
import numpy as np
ratios = []
for triplet in initial_triplets:
    filtered_triplets_count = 0
    triplets_count = 0

    if triplet in filtered_triplets:
        filtered_triplets_count = filtered_triplets[triplet]
    if triplet in triplets:
        triplets_count = triplets[triplet]  
        
    if not initial_triplets[triplet] == filtered_triplets_count + triplets_count:
        print(f"Triplet: {triplet}")
        print(f"Initial triplets count: {initial_triplets[triplet]}")
        print(f"Filtered triplets count: {filtered_triplets_count}")
        print(f"Triplets count: {triplets_count}")
        print(f"Ratio: {filtered_triplets_count / initial_triplets[triplet]}")
        print("-" * 100)
    
    ratios += [filtered_triplets_count / initial_triplets[triplet]]

np.mean(ratios), np.std(ratios)

Triplet: 4hop3__220945_88460_30152_20999
Initial triplets count: 225
Filtered triplets count: 0
Triplets count: 214
Ratio: 0.0
----------------------------------------------------------------------------------------------------
Triplet: 2hop__458672_20057
Initial triplets count: 168
Filtered triplets count: 0
Triplets count: 154
Ratio: 0.0
----------------------------------------------------------------------------------------------------
Triplet: 3hop1__178435_547811_41132
Initial triplets count: 184
Filtered triplets count: 0
Triplets count: 172
Ratio: 0.0
----------------------------------------------------------------------------------------------------
Triplet: 3hop1__823336_228453_86925
Initial triplets count: 182
Filtered triplets count: 0
Triplets count: 173
Ratio: 0.0
----------------------------------------------------------------------------------------------------
Triplet: 2hop__283_574195
Initial triplets count: 159
Filtered triplets count: 0
Triplets count: 148
Ratio: 0.0

(0.0, 0.0)

In [16]:
import numpy as np
ratios = []
for text_sample in initial_triplets:
    ontology_filtered_triplets_count = 0
    filtered_triplets_count = 0
    triplets_count = 0

    if text_sample in ontology_filtered_triplets:
        ontology_filtered_triplets_count = ontology_filtered_triplets[text_sample]
    if text_sample in filtered_triplets:
        filtered_triplets_count = filtered_triplets[text_sample]
    if text_sample in triplets:
        triplets_count = triplets[text_sample]  
        
    if initial_triplets[text_sample] == filtered_triplets_count + triplets_count + ontology_filtered_triplets_count:
        pass
        # print(f"Triplet: {triplet}")
        # print(f"Initial triplets count: {initial_triplets[text_sample]}")
        # print(f"Filtered triplets count: {filtered_triplets_count}")
        # print(f"Ontology filtered triplets count: {ontology_filtered_triplets_count}")
        # print(f"Triplets count: {triplets_count}")
        # print(f"Ratio: {ontology_filtered_triplets_count / initial_triplets[text_sample]}")
        # print("-" * 100)
    else:
        print(f"Triplet: {text_sample}")
        print(f"Initial triplets count: {initial_triplets[text_sample]}")
        print(f"Filtered triplets count: {filtered_triplets_count}")
        print(f"Ontology filtered triplets count: {ontology_filtered_triplets_count}")
        print(f"Triplets count: {triplets_count}")
        # print(f"Ratio: {ontology_filtered_triplets_count / (filtered_triplets_count + triplets_count + ontology_filtered_triplets_count)}")
        print("-" * 100)
    if (filtered_triplets_count + triplets_count + ontology_filtered_triplets_count) != 0:
        ratios += [ontology_filtered_triplets_count/ (filtered_triplets_count + triplets_count + ontology_filtered_triplets_count)]

np.mean(ratios)*100, np.std(ratios)*100

NameError: name 'ontology_filtered_triplets' is not defined

In [27]:
initial_triplets[text_sample], filtered_triplets_count, triplets_count, ontology_filtered_triplets_count

(119, 0, 114, 2)

In [28]:
for item in db.get_collection("triplets").find({'sample_id': '2hop__121145_561444', 'relation': 'creator'}, {'relation': 1, 'subject': 1, 'object': 1}):
    print(item)


In [29]:
for item in db.get_collection("triplets").find({'sample_id': '2hop__121145_561444'}, {'relation': 1}):
    print(item)


In [30]:
# db.get_collection("triplets").drop()
# db.get_collection("filtered_triplets").drop()
# db.get_collection("ontology_filtered_triplets").drop()
# db.get_collection("initial_triplets").drop()